In [ ]:
import cv2
import numpy as np
import math
# import copy

capture = cv2.VideoCapture(0)

while capture.isOpened():
    ret,frame = capture.read()

    cv2.rectangle(frame,(100,100),(400,400),(0,255,0),0)
    crop_image=frame[100:400, 100:400]

    blur = cv2.GaussianBlur(frame,(3,3),0) 
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    
    mask2 = cv2.inRange(hsv, np.array([0, 48, 70]),np.array([20, 255, 255]))
    kernel=np.ones((5,5))
    dilation= cv2.dilate(mask2, kernel, iterations = 1)
    erosion= cv2.erode(dilation, kernel, iterations = 1)
    
    filtered = cv2.GaussianBlur(erosion,(3,3),0) 
    ret1, thresh = cv2.threshold(filtered, 127, 255, 0)
    cv2.imshow("th",thresh)
    contours,_ = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    try:
        contour=max(contours,key=lambda x:cv2.contourArea(x))

        x , y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image,(x-100,y-110),(x+w-100,y+h-80),(0,0,255),0)

        cv2.putText(crop_image, 'Width{}cm'.format(w/10), (0, 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,(255, 0, 255), 2)
        cv2.putText(crop_image, 'Height{}cm'.format(h/10), (0, 70), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,(255, 0, 255), 2)  

         #make convex hull around hand
        hull = cv2.convexHull(contour)
        drawing = np.zeros(crop_image.shape, dtype= np.uint8)

        cv2.drawContours(frame,[contour],-1,(0,255,0),0)
        # cv2.drawContours(frame,[hull],-1,(0,0,255),0)

    
     #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour, hull)

        count_defects=0

        #code for finding no. of defects due to fingers
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])
            # pt= (100,180)
            
            
            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)

            angle = (math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 180)/3.14

            if angle<=90:
                count_defects+=1
               
    except:
        pass

    cv2.imshow("gesture",frame)
    all_image=np.hstack((drawing ,crop_image))
    cv2.imshow("c",all_image)   

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
# After the loop release the cap object 
capture.release() 
# Destroy all the windows 
cv2.destroyAllWindows()      